<a href="https://colab.research.google.com/github/blade-git/Hotel-Review-Sentiment-Analysis/blob/main/%EA%B0%90%EC%84%B1%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
pip install vaderSentiment

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/tripadvisor_hotel_reviews.csv')  # 파일 경로에 맞게 수정

# VADER 초기화
analyzer = SentimentIntensityAnalyzer()

# VADER 감성 분석 적용
data['VADER_Score'] = data['Review'].apply(lambda review: analyzer.polarity_scores(review)['compound'])
data['Sentiment'] = data['VADER_Score'].apply(lambda score: 'Positive' if score > 0.05 else ('Negative' if score < -0.05 else 'Neutral'))

# 결과 출력
print(data[['Review', 'VADER_Score', 'Sentiment']])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                                  Review  VADER_Score  \
0      nice hotel expensive parking got good deal sta...       0.9626   
1      ok nothing special charge diamond member hilto...       0.9870   
2      nice rooms not 4* experience hotel monaco seat...       0.9842   
3      unique, great stay, wonderful time hotel monac...       0.9920   
4      great stay great stay, went seahawk game aweso...       0.9870   
...                                                  ...          ...   
20486  best kept secret 3rd time staying charm, not 5...       0.9699   
20487  great location price view hotel great quick pl...       0.9753   
20488  ok just looks nice modern outside, desk staff ...       0.2629   
20489  hotel theft ruined vacation hotel opened sept ...       0.9696   
20490  people talking, ca n't believe excellent ratin...      -0.0003   

      Sentiment  
0      Positive  
1      Positive  
2      Positive  
3      Positive  
4      Positive  
...         ...

In [24]:
# 엑셀 파일로 저장
data[['Review', 'VADER_Score', 'Sentiment']].to_excel('VADER_Sentiment_Analysis_Results.xlsx', index=False)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 토픽모델링


In [25]:
!pip install pyLDAvis

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [26]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.lda_model
import nltk
from nltk.corpus import stopwords

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [27]:
# NLTK 불용어 리스트 다운로드
nltk.download('stopwords')
stop_words = stopwords.words('english')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/tripadvisor_hotel_reviews.csv')  # 파일 경로에 맞게 수정

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [29]:
# 1. 텍스트 전처리
# 리뷰 텍스트를 소문자로 변환하고, 불용어 제거 및 토큰화
data['Cleaned_Review'] = data['Review'].str.lower().replace('[^\w\s]', '', regex=True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence '\w'
<>:3: DeprecationWarning: invalid escape sequence '\w'
<ipython-input-29-aca5e5c0fa7b>:3: DeprecationWarning: invalid escape sequence '\w'
  data['Cleaned_Review'] = data['Review'].str.lower().replace('[^\w\s]', '', regex=True)


In [30]:
# 2. 문서-단어 행렬 생성
# CountVectorizer를 사용해 텍스트 데이터를 벡터화
vectorizer = CountVectorizer(stop_words=stop_words, max_features=1000)
doc_term_matrix = vectorizer.fit_transform(data['Cleaned_Review'])

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [31]:
# 3. LDA 모델 학습
# 주제 수는 임의로 5로 설정 (필요 시 변경 가능)
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_model.fit(doc_term_matrix)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=5, random_state=42)

In [32]:
# 4. 각 주제에 포함된 단어 출력
words = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx + 1}: ", [words[i] for i in topic.argsort()[-10:]])

Topic 1:  ['room', 'service', 'area', 'good', 'food', 'nice', 'great', 'beach', 'hotel', 'pool']
Topic 2:  ['like', 'staff', 'rooms', 'service', 'desk', 'night', 'stay', 'nt', 'hotel', 'room']
Topic 3:  ['rooms', 'helpful', 'friendly', 'stayed', 'room', 'location', 'stay', 'staff', 'great', 'hotel']
Topic 4:  ['rooms', 'clean', 'walk', 'nice', 'breakfast', 'great', 'location', 'good', 'room', 'hotel']
Topic 5:  ['room', 'day', 'people', 'time', 'great', 'good', 'food', 'nt', 'beach', 'resort']


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [33]:
# 5. 각 주제에 포함된 단어 출력
words = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    print(f"Topic {topic_idx + 1}: ", [words[i] for i in topic.argsort()[-10:]])

Topic 1:  ['room', 'service', 'area', 'good', 'food', 'nice', 'great', 'beach', 'hotel', 'pool']
Topic 2:  ['like', 'staff', 'rooms', 'service', 'desk', 'night', 'stay', 'nt', 'hotel', 'room']
Topic 3:  ['rooms', 'helpful', 'friendly', 'stayed', 'room', 'location', 'stay', 'staff', 'great', 'hotel']
Topic 4:  ['rooms', 'clean', 'walk', 'nice', 'breakfast', 'great', 'location', 'good', 'room', 'hotel']
Topic 5:  ['room', 'day', 'people', 'time', 'great', 'good', 'food', 'nt', 'beach', 'resort']


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# 주제별 감성 분석

In [34]:
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [35]:
# 데이터 불러오기
data = pd.read_csv('/content/drive/MyDrive/tripadvisor_hotel_reviews.csv')  # 파일 경로에 맞게 수정
data['Cleaned_Review'] = data['Review'].str.lower().replace('[^\w\s]', '', regex=True)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:3: DeprecationWarning: invalid escape sequence '\w'
<>:3: DeprecationWarning: invalid escape sequence '\w'
<ipython-input-35-f06f2317131b>:3: DeprecationWarning: invalid escape sequence '\w'
  data['Cleaned_Review'] = data['Review'].str.lower().replace('[^\w\s]', '', regex=True)


In [36]:
# 토픽 모델링 수행 (LDA)
vectorizer = CountVectorizer(stop_words=stop_words, max_features=1000)
doc_term_matrix = vectorizer.fit_transform(data['Cleaned_Review'])
lda_model = LatentDirichletAllocation(n_components=5, random_state=42)
lda_topics = lda_model.fit_transform(doc_term_matrix)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
# 각 리뷰에 대해 가장 높은 비중의 주제를 할당
data['Topic'] = lda_topics.argmax(axis=1)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [38]:
# VADER 감성 분석 초기화
analyzer = SentimentIntensityAnalyzer()

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [39]:
# 각 리뷰에 대해 VADER 감성 분석 수행 및 감정 분류
def get_sentiment(score):
    if score > 0.05:
        return 'Positive'
    elif score < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

data['VADER_Score'] = data['Review'].apply(lambda review: analyzer.polarity_scores(review)['compound'])
data['Sentiment'] = data['VADER_Score'].apply(get_sentiment)

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [40]:
# 주제별 감정 분석 결과 요약
topic_sentiment_summary = data.groupby(['Topic', 'Sentiment']).size().unstack(fill_value=0)
topic_sentiment_summary['Total'] = topic_sentiment_summary.sum(axis=1)
topic_sentiment_summary['Positive_Percentage'] = (topic_sentiment_summary['Positive'] / topic_sentiment_summary['Total']) * 100
topic_sentiment_summary['Negative_Percentage'] = (topic_sentiment_summary['Negative'] / topic_sentiment_summary['Total']) * 100

# 결과 출력
print(topic_sentiment_summary)

Sentiment  Negative  Neutral  Positive  Total  Positive_Percentage  \
Topic                                                                
0                52        5      1359   1416            95.974576   
1              1051       61      2449   3561            68.772817   
2                36        6      5377   5419            99.224949   
3               179       19      6643   6841            97.105686   
4               179        7      3068   3254            94.283958   

Sentiment  Negative_Percentage  
Topic                           
0                     3.672316  
1                    29.514181  
2                     0.664329  
3                     2.616577  
4                     5.500922  


/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
